# Model quality check and improvement workflow

1.Download the initila model constructed from ModelSEED. The format of initial model is SBML file.


2.generate an excel file for the model for easy checking. 

In [2]:
#2 excel format model generation. for checking and revising model, such as reactions and their lower or upper bound information.
from metnet import sbml_excel
inputforder='./data/'
sbml_excel(inputforder+'Seed1111.5.65232.xml','Seed1111.5.65232.xlsx')# sbml to excel
#sbmltoexcel.model_excel(model_1,'model_1.xlsx') # cobrapy model to excel. If we want to check the model we can change it to excel formate at any time

3 initial check and model revision

In [1]:
#3.1 import some modules and read the sbml model
from __future__ import print_function
import cobra
import re
import pandas as pd
from cobra import Model, Reaction, Metabolite
from cobra.flux_analysis import pfba
inputforder='/media/jupyter/data/'
model = cobra.io.read_sbml_model(inputforder+'Seed1111.5.65232.xml') # read sbml format model

In [2]:
#3.2 Calculate optimal growth rate and check its input and output
model.optimize()
model.summary() #check input/outputs for the optimal solution
#model.medium
# many substrates can be consumed, leading to unreasonable high growth rates, need to modify the boundaries of exchange reactions

IN FLUXES             OUT FLUXES            OBJECTIVES
--------------------  --------------------  -------------
cpd00794_e0    1e+03  cpd00013_e0    1e+03  biomass0  149
cpd00162_e0    1e+03  cpd00001_e0    1e+03
cpd00129_e0  692      cpd00011_e0    1e+03
cpd00130_e0  692      cpd00067_e0    1e+03
cpd00007_e0  478      cpd00221_e0    1e+03
cpd00307_e0  298      cpd00797_e0  969
cpd00100_e0  279      cpd00027_e0  608
cpd00075_e0  140      cpd00023_e0  571
cpd00009_e0  121      cpd00036_e0  352
cpd00107_e0   56      cpd00092_e0  267
cpd00039_e0   46.3    cpd00082_e0  261
cpd00226_e0   45.1    cpd00156_e0  189
cpd15606_e0   37.3    cpd00069_e0   19.3
cpd00051_e0   36.8
cpd00322_e0   36.1
cpd00276_e0   35.2
cpd00105_e0   30.9
cpd11581_e0   29.9
cpd00066_e0   23
cpd11591_e0   19.5
cpd01017_e0   12.7
cpd01080_e0   12.7
cpd00119_e0   11.8
cpd00065_e0    7.04
cpd03847_e0    3.73
cpd10515_e0    1.85
cpd00393_e0    1.39
cpd11606_e0    0.924
cpd00048_e0    0.462
cpd00030_e0    0.462
cpd00058_e0 

In [3]:
#3.3 Change boundary of exchange reactions
'''
1. mark exchange reactions in excel file, mark exchange reactions for carbon,oxygen,nitrogen ions etc
2. normally only allow one carbon source (default was glucose), uptake rate was set at 10mmol/g/h.
3. oxygen exchange rate can be set to 0 for anaerobic or unlimited for aerobic
4. CO2 exchange rate can be negative for CO2 fixation or 0 so that CO2 can only be released
5. nitrogen (NH3) sulfur (H2S2O3,sulfate) and phosphate (Pi) sources can also be set, normally not limited
6. exchange reactions for ions(Pb,Cd2,Cu2,Fe2,Fe3,Mg,K,Zn2,Na,Co2,Cl,Hg2,Mn2,Ca2) and H2O were unconstrained.
7. boundaries of all other exchange reactions were set to 0-1000 so that the metabolites can only be produced but not consumed. 
'''
c_exchange=['EX_cpd00027_e0'] #exchange reaction for carbon sources, use list to allow multiple sources
for ex in c_exchange:
    r=model.reactions.get_by_id(ex)
    r.bounds=(-10, 1000) # maximal uptake rate 10mmol/g/h
o_exchange=['EX_cpd00007_e0'] #exchange reaction for oxygen for anaerobic or aerobic conditions
for ex in o_exchange:
    r=model.reactions.get_by_id(ex)
    r.bounds=(-1000, 1000) 
co2_exchange=['EX_cpd00011_e0'] #exchange reaction for CO2, 
for ex in co2_exchange:
    r=model.reactions.get_by_id(ex)
    r.bounds=(0, 1000) 
n_exchange=['EX_cpd00013_e0'] #exchange reaction for N sources
for ex in n_exchange:
    r=model.reactions.get_by_id(ex)
    r.bounds=(-1000, 1000) 
s_exchange=['EX_cpd00048_e0','EX_cpd00268_e0'] #exchange reaction for sulafte and H2S2O3
for ex in s_exchange:
    r=model.reactions.get_by_id(ex)
    r.bounds=(-1000, 1000) 
p_exchange=['EX_cpd00009_e0','EX_cpd00012_e0'] #exchange reaction for Pi and PPi
for ex in p_exchange:
    r=model.reactions.get_by_id(ex)
    r.bounds=(-1000, 1000) 
ion_exchange=['EX_cpd04097_e0','EX_cpd00001_e0','EX_cpd00058_e0','EX_cpd10515_e0','EX_cpd10516_e0','EX_cpd00254_e0','EX_cpd00205_e0',
            'EX_cpd00034_e0','EX_cpd00971_e0','EX_cpd00149_e0','EX_cpd00099_e0','EX_cpd00531_e0','EX_cpd00030_e0','EX_cpd00063_e0']
for ex in ion_exchange:
    r=model.reactions.get_by_id(ex)
    r.bounds=(-1000, 1000) 
preset_exchange=c_exchange+o_exchange+co2_exchange+n_exchange+s_exchange+p_exchange+ion_exchange
for rea in model.boundary: 
    if rea.id not in preset_exchange: #all other exchange reastions can only be outputs
        rea.bounds=(0,1000)
model.optimize()
model.summary() 
cobra.io.write_sbml_model(model, "model01.xml")
#growth likely to be zero, gaps in the network, but first check ATP production

IN FLUXES        OUT FLUXES       OBJECTIVES
---------------  ---------------  -----------------
cpd00007_e0  60  cpd00001_e0  60  biomass0  4.7e-14
cpd00027_e0  10  cpd00011_e0  60


4 Check and revise the ATP production pathway 

In [4]:
#4.1 
'''
we often get very high ATP production rates caused by wrong ATP generation cycle, need to check the pathways one by one to correct
the mistakes using the ATP consumption reaction (maintenance) rxn00062 as objective reaction
find the maintenance reaction by searching in excel, should change the lower bound to maintenace coefficient such as 4.9 
'''
#rin=model.reactions.get_by_id("EX_cpd00027_e0") # check the glucose input
#rin.bounds=(0, 1000) # we can turn off the carbon input for checking wrong ATP generation pathways
if "rxn00062_c0" not in model.reactions: # rxn00062_c0 is the maintenance reaction of the model, if rxn00062_c0 not in model, add this reaction in model
    #reaction = Reaction('ATPM')
    reaction = Reaction('rxn00062_c0')
    model.add_reactions([reaction])
    reaction.name = 'ATP maintenance reaction'
    reaction.build_reaction_from_string('cpd00001_c0 + cpd00002_c0 => cpd00008_c0 + cpd00009_c0 + cpd00067_c0')
    model.add_reactions([reaction])
#else:
    #model.reactions.get_by_id("rxn00062_c0").id='ATPM'
atpm=model.reactions.get_by_id("rxn00062_c0")
atpm.bounds=(4.9, 1000) #maintenace rate as lower bound
#atpm.bounds=(-1000, 1000) #修改以使其可最小化净生成ATP，从而可计算净消耗ATP的无效循环
model.objective=atpm # change objective function
model.optimize()
model.summary() 

IN FLUXES        OUT FLUXES       OBJECTIVES
---------------  ---------------  ------------------
cpd00007_e0  60  cpd00001_e0  60  rxn00062_c0  1e+03
cpd00027_e0  10  cpd00011_e0  60


In [5]:
#4.2 output the pathway to a text file to find the mistakes
from metnet import pathway
outputfile_name ='atp.txt' # provide output file name
fluxes = pfba(model).fluxes #use pfba to get a simple path，you should provide your model name
pathway(model,fluxes,outputfile_name)

By checking the result pathway file, we can find ATP without consumption of any substrate but from a cycle pathway. Reaction flux for rxn00058_c0 ((4.0)H_c0 + (4.0)Cytochrome_c2_c0 + O2_c0 -->(2.0)H2O_c0 + (4.0)Cytochrome_c3_c0) is -500, however this reaction	should be irreversible as O2_c0 should only be consumed but not produced in the respiratory chain. This should be corrected by change the lower bound of this reaction to zeor to make it irreversible. 
Actually most errors in ATP generation were caused by mistakes in reaction reversibility, we need to repeat the process untill all erros are corrected and a reasonable amount of ATP is produced from the carbon source.We can use of a set of rules to quickly determine which reaction in a wrong pathway should be set to irreversible.
1.	Oxygen consumption reactions,
2.	Most of the carbon dioxide production reaction (CO2 is only used as substrate when a high energy substrate such as PEP (phophoenolpyruvate) and ATP is consumed at the same time),
3.	Most of the NH3 production reaction (NH3 is only used in two NH3 assimilation reactions and carbamoyl phosphate production),
4.	Most of the phosphate production reaction (the reaction is regarded as reversible only when phosphate reacts with another high energy substrate such as AcCoA)
5.	Reactions in which S-Adenosyl-L-methionine is converted to S-Adenosyl-L-homocystine for providing a methyl group,
6.	Reactions in which tetrahydrofolate (THF) is produced for transferring one carbon unit, 
7.	Most of the ATP (or other high energy metabolites) consumption reactions, except for reactions with another high energy metabolite such as GTP, CAP, acetyl phosphate, Acyl-CoA,
8.	UDP-sugar consumption reactions for transferring sugar units,  
9.	CDP-diacylglycerol consumption reactions for phosphatidyl group transfer,
10.	reactions like: 3'-Phosphoadenylylsulfate (PAPS) + A = adenosine 3',5'-bisphosphate (PAP) + B

To avoid the time consuming correction process, we compiled a list of reactions from ModelSeed that should be corrected to irreversible to avoid wrong ATP generation pathways. The model can be updated from this list. 
请生成一个修正反应列表，文本文件含三列数据 反应ID，下界 上界，由此文件更新sbml文件的model02.


In [6]:
#4.3 Change the boundary of the model according to this list , then wo got the model 2.0 version
import csv
s = csv.reader(open(inputforder+'PST_for_ATP_list.txt'), delimiter="\t")
realist = []
realist.extend(s)
for j in realist:
    model.reactions.get_by_id(j[0]).bounds=(float(j[1]),float(j[2]))
cobra.io.write_sbml_model(model, "model02.xml")
# After running this step, go to run 4.2 to check the pathway of ATP synthesis.

5 When the ATP synthetic pathway was right, we go on check the biomass synthesis.

In [7]:
#5.1 Check the optimal growth rate of this model.
model.objective=model.reactions.get_by_id('biomass0') # change objective function
model.optimize()
model.summary() 

IN FLUXES          OUT FLUXES          OBJECTIVES
-----------------  ------------------  -------------------
cpd00007_e0  1.22  cpd00001_e0  2.7    biomass0  -7.03e-17
cpd00027_e0  0.98  cpd00011_e0  1.96
cpd00013_e0  0.49  cpd00067_e0  0.735
                   cpd00107_e0  0.49
                   cpd00142_e0  0.245


In [8]:
#5.2 If the growth rate was 0, calculate every biomass precursor to check which matabolite can not be generated
# mark exchange reactions in excel file, mark exchange reactions for carbon,oxygen,nitrogen ions etcexclude_mets contain all metabolites that can be directly exculded by this model. 
from metnet import calculate, mkdir
#import makefolder
#import biomass_cal
outfolder = './biomass_precursors/' # (*Should provide) Folder's for store all calculated results of biomass precursors
mkdir (outfolder) # Create Folder
summary = outfolder + 'summary.txt'
#exclude_mets=['cpd00254_c0','cpd00205_c0','cpd17042_c0','cpd00034_c0','cpd00063_c0','cpd17041_c0','cpd00099_c0','cpd00149_c0','cpd17043_c0','cpd00030_c0','cpd00048_c0','cpd10515_c0','cpd10516_c0','cpd00058_c0','cpd00001_c0']
precursors=[]
for i in model.reactions.get_by_id('biomass0').reactants: # The biomass ID of initial model usually contains 'bio'
    #if str(i) not in exclude_mets:
        precursors.append(i)
calculate(precursors,model,outfolder,summary) # Calculate all precursors' optimal production rtae

./biomass_precursors/ 目录已存在


In [12]:
#5.3 Calculate the biomass precursors of manually corrected model to check synthetic pathways of these nonsynthetic metabolite
from metnet import calculate, mkdir
import csv
import cobra
from cobra import Model, Reaction, Metabolite
outfolder = '/media/jupyter/data/' # (*Should provide) Folder's for store all calculated results of biomass precursors
mkdir (outfolder) # Create Folder
summary = outfolder + 'summary_pstmodel.txt'
modelpst = cobra.io.read_sbml_model(outfolder+'PST_model.xml') # read sbml format model
reaction = Reaction('bio')
modelpst.add_reactions([reaction])
reaction.build_reaction_from_string('35.5403092430435 H2O_c0 + 40.1101757365074 ATP_c0 + 0.00309646685192537 NAD_c0 + 0.00309646685192537 NADP_c0 + 0.00309646685192537 CoA_c0 + 0.00309646685192537 FAD_c0 + 0.00309646685192537 Pyridoxal_phosphate_c0 + 0.00309646685192537 S_Adenosyl_L_methionine_c0 + 0.219088153012743 L_Glutamate_c0 + 0.00309646685192537 Heme_c0 + 0.00309646685192537 Mn2_c0 + 0.509869786991038 Glycine_c0 + 0.00309646685192537 Zn2_c0 + 0.427934380173264 L_Alanine_c0 + 0.135406821203723 GTP_c0 + 0.285438020490179 L_Lysine_c0 + 0.200830806928348 L_Aspartate_c0 + 0.00309646685192537 GSH_c0 + 0.00309646685192537 Sulfate_c0 + 0.246696822701341 L_Arginine_c0 + 0.219088153012743 L_Glutamine_c0 + 0.179456352975885 L_Serine_c0 + 0.00309646685192537 TPP_c0 + 0.00309646685192537 Cu2_c0 + 0.127801422590767 L_Methionine_c0 + 0.0908319049068452 UTP_c0 + 0.00309646685192537 Ca2_c0 + 0.0472019191450218 L_Tryptophan_c0 + 0.154519490031345 L_Phenylalanine_c0 + 0.120676604606612 L_Tyrosine_c0 + 0.0761464922056484 L_Cysteine_c0 + 0.00309646685192537 Tetrahydrofolate_c0 + 0.00309646685192537 Cl__c0 + 0.375388847540127 L_Leucine_c0 + 0.0320579110651499 dATP_c0 + 0.00309646685192537 Putrescine_c0 + 0.0792636000737159 L_Histidine_c0 + 0.184354665339991 L_Proline_c0 + 0.200830806928348 L_Asparagine_c0 + 0.00309646685192537 Co2_c0 + 0.352233189091625 L_Valine_c0 + 0.211072732780569 L_Threonine_c0 + 0.00309646685192537 Calomide_c0 + 0.00309646685192537 10_Formyltetrahydrofolate_c0 + 0.00309646685192537 K_c0 + 0.00309646685192537 Riboflavin_c0 + 0.00309646685192537 Mg_c0 + 0.00309646685192537 Spermidine_c0 + 0.241798510337235 L_Isoleucine_c0 + 0.00309646685192537 5_Methyltetrahydrofolate_c0 + 0.0320579110651499 TTP_c0 + 0.00309646685192537 Siroheme_c0 + 0.0250105977108944 Bactoprenyl_diphosphate_c0 + 0.00309646685192537 Fe2_c0 + 0.00309646685192537 fe3_c0 + 0.00309646685192537 ACP_c0 + 0.00309646685192537 2_Demethylmenaquinone_8_c0 + 0.0250105977108944 core_oligosaccharide_lipid_A_c0 + 0.00309646685192537 Menaquinone_8_c0 + 0.0106480421341882 phosphatidylethanolamine_dioctadecanoyl_c0 + 0.0106480421341882 Phosphatidylglycerol_dioctadecanoyl_c0 + 0.00309646685192537 Ubiquinone_8_c0 + 0.0250105977108944 Peptidoglycan_polymer_n_subunits_c0 + 0.0106480421341882 Diisoheptadecanoylphosphatidylethanolamine_c0 + 0.0106480421341882 Dianteisoheptadecanoylphosphatidylethanolamine_c0 + 0.0106480421341882 Diisoheptadecanoylphosphatidylglycerol_c0 + 0.0106480421341882 Dianteisoheptadecanoylphosphatidylglycerol_c0 + 0.0106480421341882 Stearoylcardiolipin_B_subtilis_c0 + 0.0106480421341882 Isoheptadecanoylcardiolipin_B_subtilis_c0 + 0.0106480421341882 Anteisoheptadecanoylcardiolipin_B_subtilis_c0 + Protein_biosynthesis_c0 + DNA_replication_c0 + RNA_transcription_c0 --> 40.0 ADP_c0 + 39.9969035331481 Phosphate_c0 + 0.484633900402731 PPi_c0 + 40.0 H_c0 + 0.00309646685192537 Dimethylbenzimidazole_c0 + 0.00309646685192537 Cobinamide_c0 + Biomass_c0 + 0.00309646685192537 apo_ACP_c0 + 0.0250105977108944 Peptidoglycan_polymer_n_1_subunits_c0')
precursors=[]
for i in modelpst.reactions.get_by_id('bio').reactants:     
    precursors.append(i)
#biomass_cal.calculate(precursors,model,outfolder,summary) # Calculate all precursors' optimal production rtae
writefile = open(summary, 'w')
with modelpst:
    for i in precursors:
        demand = modelpst.add_boundary(i,type='demand')
        modelpst.objective = demand
        outputfile_name = outfolder + str(i) + '.txt'
        fluxes = cobra.flux_analysis.pfba(modelpst).fluxes
        a='DM_'+ str(i)
        obj=str(fluxes[a])
        writefile.write(str(i)+"\t"+obj+"\n")                  
        flux = open(outputfile_name, 'w')
        for r,v in fluxes.iteritems():
            if abs(v)>1e-6:
            #print (r,v)
                for line in modelpst.reactions:
                    if line.id==r:
                        m=str(modelpst.reactions.get_by_id(r))
                        n=m.split(':')
                        flux.write(r +'\t' + str(round(v,4)) + '\t' + n[1] +'\n')
        flux.close()
writefile.close()

/media/jupyter/data/ 目录已存在
unknown metabolite 'Peptidoglycan_polymer_n_subunits_c0' created
unknown metabolite 'Protein_biosynthesis_c0' created
unknown metabolite 'Peptidoglycan_polymer_n_1_subunits_c0' created


In [16]:
#5.4 Calculate all matabolites associated with unsynthetic precursors.
from metnet import calculate, mkdir
outfolder = './Arg_mets/' # (*Should provide) Folder's for store all calculated results of biomass precursors
mkdir (outfolder) # Create Folder
summary = outfolder + 'summary_Argmets.txt'
Arg_met=['cpd00051_c0','cpd02152_c0','cpd00041_c0','cpd00146_c0','cpd00274_c0','cpd00064_c0','cpd00477_c0','cpd00342_c0','cpd00918_c0','cpd02552_c0','cpd00023_c0','cpd00024_c0','cpd03187_c0','cpd00260_c0','cpd00137_c0','cpd00032_c0','cpd00061_c0','cpd00482_c0','cpd00169_c0','cpd00203_c0','cpd00102_c0','cpd02711_c0','cpd00284_c0','cpd00911_c0','cpd00079_c0','cpd00027_c0']
calculate(Arg_met,model,outfolder,summary) # Calculate all precursors' optimal production rtae

./Arg_mets/ 目录已存在
